In [1]:
%%file romberg.f90
    SUBROUTINE polint(xx, yy, n, x, y, err, c, d)
    ! Routine to find one interpolated value given a table of n values
    implicit none

    ! Inputs and outputs
    integer, intent(in) :: n
    real*8, dimension(n), intent(in) :: xx, yy
    real*8, intent(in) :: x
    real*8, intent(out) :: y, err
    real*8, dimension(n), intent(out) :: c, d

    ! Local variables
    integer :: i, m, ns
    real*8 :: den, dif, dift, ho, hp, w
    ns = 1
    dif = ABS(x - xx(1))

    ! Find the nearest point
    do i = 1, n
        dift = ABS(x - xx(i))
        if (dift < dif) then
            ns = i
            dif = dift
        end if
        c(i) = yy(i)
        d(i) = yy(i)
    end do

    y = yy(ns)
    ns = ns - 1

    ! Calculate polynomial interpolation
    do m = 1, n - 1
        do i = 1, n - m
            ho = xx(i) - x
            hp = xx(i + m) - x
            w = c(i + 1) - d(i)
            den = ho - hp
            if (den == 0.0d0) pause 'Failure in polint: denominator is zero'
            den = w / den
            d(i) = hp * den
            c(i) = ho * den
        end do
        if (2 * ns < n - m) then
            err = c(ns + 1)
        else
            err = d(ns)
            ns = ns - 1
        end if
        y = y + err
    end do

    return
    end subroutine polint


    SUBROUTINE trapzd(func, a, b, sin, sout, n)
    ! Routine to perform the trapezoidal rule for numerical integration
    implicit none

    ! Inputs and outputs
    real*8, intent(in) :: a, b, sin
    real*8, intent(out) :: sout
    integer, intent(in) :: n
    external :: func

    ! Local variables
    integer :: it, j
    real*8 :: del, sum, tnm, x

    if (n == 1) then
        sout = 0.5d0 * (b - a) * (func(a) + func(b))
    else
        it = 2**(n - 2)
        tnm = it
        del = (b - a) / tnm
        x = a + 0.5d0 * del
        sum = 0.d0

        do j = 1, it
            sum = sum + func(x)
            x = x + del
        end do

        sout = 0.5d0 * (sin + (b - a) * sum / tnm)
    end if

    return
    end subroutine trapzd


    SUBROUTINE qromb(func, a, b, ss, dss, numcalls, EPS, K, c, d)
    ! Routine to perform Romberg integration
    implicit none

    ! Inputs and outputs
    real*8, intent(out) :: ss, dss
    integer, intent(out) :: numcalls
    real*8, optional, intent(in) :: EPS
    integer, optional, intent(in) :: K
    real*8, dimension(K), intent(hide) :: c, d
    real*8, intent(in) :: a, b

    external :: func

    ! Local variables
    integer :: JMAX, JMAXP, KM, j
    real*8 :: h(JMAXP), s(JMAXP), sold
    real*8 :: y1, y2

    ! Parameters
    parameter (EPS = 1.e-6, JMAX = 20, JMAXP = JMAX + 1, K = 5, KM = K - 1)
    h(1) = 1.d0
    KM = K - 1
    sold = 0.0d0

    ! Romberg integration loop
    do j = 1, JMAX
        call trapzd(func, a, b, sold, s(j), j)
        if (j >= K) then
            call polint(h(j - KM), s(j - KM), K, 0.d0, ss, dss, c, d)
            if (ABS(dss) <= EPS * ABS(ss)) then
                numcalls = 2**(j - 1) + 1
                return
            end if
        end if
        sold = s(j)
        h(j + 1) = 0.25d0 * h(j)
    end do

    ! No convergence
    numcalls = -1
    return
    end subroutine qromb


Writing romberg.f90


In [2]:
import os ,sys

In [3]:
!python -m numpy.f2py -c romberg.f90 -m romberg

running build
running config_cc
INFO: unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
INFO: unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
INFO: build_src
INFO: building extension "romberg" sources
INFO: f2py options: []
INFO: f2py:> C:\Users\bprav\AppData\Local\Temp\tmp8qnvu50b\src.win-amd64-3.9\rombergmodule.c
creating C:\Users\bprav\AppData\Local\Temp\tmp8qnvu50b\src.win-amd64-3.9
Reading fortran codes...
	Reading file 'romberg.f90' (format:free)
Line #112 in romberg.f90:"    parameter (EPS = 1.e-6, JMAX = 20, JMAXP = JMAX + 1, K = 5, KM = K - 1) "
	param_eval: got "eval() arg 1 must be a string, bytes or code object" on 4
Line #112 in romberg.f90:"    parameter (EPS = 1.e-6, JMAX = 20, JMAXP = JMAX + 1, K = 5, KM = K - 1) "
	param_eval: got "eval() arg 1 must be a string, bytes or code object" on 4
Line #112 in romberg.f90:"    parameter (EPS = 1.e-6, JMAX = 20, JMAXP = JMA

d:\Anaconda\lib\site-packages\numpy\f2py\f2py2e.py:719: VisibleDeprecationWarning: distutils has been deprecated since NumPy 1.26.xUse the Meson backend instead, or generate wrapperswithout -c and use a custom build script
  builder = build_backend(
append_needs: unknown need 'double'
routsign2map: Confused: function qromb has externals ['func'] but no "use" statement.
sign2map: Confused: external func is not in lcb_map[].
append_needs: unknown need 'func'
append_needs: unknown need 'func'
d:\Anaconda\lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ***********************************************

In [ ]:
romberg